In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Customer Price Index simulation using Monte Carlo method

### Overview

#### Consumer Price Index (CPI)

The **Consumer Price Index (CPI)** is a measure of the average change in prices paid by consumers for a basket of goods and services. **Inflation** is the rate at which the general level of prices for goods and services is rising and, consequently, the purchasing power of currency is falling.

The **CPI** is a key measure of inflation because it tracks the prices of a wide range of goods and services that are purchased by consumers. If the CPI increases, it means that the prices of goods and services have increased on average. This is a sign of inflation.

For example, if the **CPI increases by 2% in a year**, it means that the **average price of goods and services has increased by 2% over that year**. This means that consumers will need to spend 2% more money to buy the same basket of goods and services that they bought the previous year.

The CPI is used by a variety of economic actors, including governments, businesses, and consumers. Governments use the CPI to set economic policy, such as interest rates. Businesses use the CPI to make pricing decisions. Consumers use the CPI to track the cost of living.

#### Monte Carlo method

This code uses the **Monte Carlo** method to simulate the future path of the **Consumer Price Index (CPI)**. The code begins by calculating the historical mean and standard deviation of the percentage change in the CPI. These values are then used to generate a large number of **simulated paths for the future CPI**. Each simulated path is generated by starting with the **current value of the CPI and then adding a series of normally distributed random increments**. The size of the random increments is determined by the historical **mean and standard deviation of the percentage change in the CPI**.

Once the simulated paths have been generated, they are used to calculate the **probability of various future events**. For example, the code can be used to calculate the **probability that the inflation rate in a given year will be below a certain target**. The **Monte Carlo** method is a **powerful tool for forecasting** future economic conditions. By simulating a large number of possible future paths, the Monte Carlo method can provide a more accurate picture of the range of possible outcomes than traditional forecasting methods.

The concepts and theory related to the Monte Carlo method are based on the theory of **probability and statistics**. The Monte Carlo method works by **randomly sampling from a probability distribution**. In the case of the CPI, the probability distribution is the distribution of the percentage change in the CPI. By randomly sampling from this distribution, the Monte Carlo method can generate a large number of possible future paths for the CPI.

The Monte Carlo method is a versatile tool that can be used to forecast a wide range of economic variables. In addition to forecasting the CPI, the Monte Carlo method can be used to forecast interest rates, exchange rates, and stock prices. The Monte Carlo method is a valuable tool for anyone who is interested in forecasting future economic conditions.

- This code utilizes the **Monte Carlo** simulation technique to model the future path of the **Consumer Price Index (CPI)** and estimate the probability of inflation falling below a specific threshold.
- It employs the **geometric Brownian motion model**, a stochastic process commonly used to represent financial asset prices, to simulate numerous potential trajectories of the CPI.
- The code extracts historical CPI data, calculates essential parameters like mean and standard deviation, and generates random Wiener increments to simulate the unpredictable nature of market movements.
- It then constructs a DataFrame of simulated CPI paths and visualizes the 80th, 50th, and 20th percentiles of these paths.
- Finally, it estimates the probability of inflation falling below 2% in a specific year by analyzing the simulated inflation rates.

#### Pandas API on Spark


The code is using the Pandas API on Spark. This is a feature of PySpark that allows users to write code using the familiar Pandas DataFrame API on Spark DataFrames. This can be a significant advantage for users who are already familiar with Pandas, as it allows them to use their existing knowledge to work with Spark DataFrames.

The Pandas API on Spark is implemented by converting Spark DataFrames to Pandas DataFrames under the hood. This means that all of the Pandas DataFrame methods are available for use on Spark DataFrames. However, it is important to note that not all Pandas methods are supported by the Pandas API on Spark. A list of supported methods can be found in the PySpark documentation.

The Pandas API on Spark can be a useful tool for users who want to take advantage of the power of Spark while still using the familiar Pandas API. This can be especially useful for users who are already familiar with Pandas and who do not want to learn the Spark DataFrame API.

In the specific case of the code being analyzed, the Pandas API on Spark is being used to generate a large number of simulated paths for the future CPI. This is done by using the pandas.DataFrame.pct_change() method to calculate the percentage change in the CPI. The percentage change is then used to generate a series of normally distributed random increments. These random increments are then added to the current value of the CPI to generate a simulated path for the future CPI.

### Setup

Since we are using the [Pandas API on Spark](https://spark.apache.org/docs/latest/api/python/user_guide/pandas_on_spark/index.html) make sure your runtime uses Spark version >= 3.2.0

#### Identity and Access Management (IAM)

Make sure the service account running this notebook has the required permissions:

- **Run the notebook**
  - AI Platform Notebooks Service Agent
  - Notebooks Admin
  - Vertex AI Administrator
- **Read files from bucket**
  - Storage Object Viewer
- **Run Dataproc jobs**
  - Dataproc Service Agent
  - Dataproc Worker

In [ ]:
%pip install pyspark==3.5.0 google-spark-connect google-cloud-dataproc pandas -q

In [ ]:
PROJECT_ID = "YOUR_PROJECT_ID"
LOCATION = "YOUR_LOCATION"

In [ ]:
#### Import dependencies
import numpy as np
import pandas as pd
from google.cloud import dataproc_v1

from pyspark.sql.types import DoubleType, IntegerType
from pyspark.sql.functions import col, year, lag, lit
from pyspark.sql.window import Window

import google.api_core.exceptions

from google.cloud.dataproc_v1 import Session, SparkConnectConfig
from google.cloud.spark_connect import GoogleSparkSession

import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

In [ ]:
def create_session_template(serverless_spark_template_id):    

    full_template_name = f"projects/{PROJECT_ID}/locations/{LOCATION}/sessionTemplates/{serverless_spark_template_id}"

    template = dataproc_v1.SessionTemplate()
    template.description = "A standard template for interactive PySpark sessions."
    template.spark_connect_session = {}
    template.name = full_template_name

    client_options = {"api_endpoint": f"{LOCATION}-dataproc.googleapis.com:443"}
    client = dataproc_v1.SessionTemplateControllerClient(
            client_options=client_options
        )
    parent = client.common_location_path(PROJECT_ID, LOCATION)
    
    try:
        request = dataproc_v1.CreateSessionTemplateRequest(
            parent=parent,
            session_template=template
        )
        result = client.create_session_template(request=request)
        print(f"Session template created: {result.name}")
    except google.api_core.exceptions.AlreadyExists:
        print(f"Session template '{serverless_spark_template_id}' already exists.")
        request = dataproc_v1.GetSessionTemplateRequest(name=full_template_name)
        result = client.get_session_template(request=request)
        print(f"Using existing session template: {result.name}")

In [ ]:
serverless_spark_template_id = "default_spark_session_template"
create_session_template(serverless_spark_template_id)

In [ ]:
session_config = Session()
session_config.spark_connect_session = SparkConnectConfig()
session_config.session_template = f"projects/{PROJECT_ID}/locations/{LOCATION}/sessionTemplates/{serverless_spark_template_id}"
spark = GoogleSparkSession.builder.projectId(PROJECT_ID).location(LOCATION).googleSessionConfig(session_config).getOrCreate()
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', 'true')

In [ ]:
raw_dataset = spark.read.option("header",True).csv("gs://dataproc-metastore-public-binaries/us_customer_price_index_yearly/")

raw_dataset = raw_dataset.withColumn("Year", col("Year").cast(IntegerType())) \
                         .withColumn("CPI", col("CPI").cast(DoubleType()))

### Pre-process dataset / filter values

In [ ]:
import warnings
from pyspark.pandas.utils import PandasAPIOnSparkAdviceWarning

warnings.filterwarnings('ignore', message='.*No Partition Defined for Window operation.*')
warnings.filterwarnings('ignore', category=PandasAPIOnSparkAdviceWarning)

cpi_80 = raw_dataset[raw_dataset.Year >= 1980]
cpi_80 = cpi_80.pandas_api(index_col='Year')

inf = cpi_80.pct_change().dropna()

### Run Monte Carlos simulation

**Formula:**  CPI_t = CPI_0 * exp((μ - 0.5 * σ^2) * t + σ * W_t)

where:

- CPI_t is the CPI at time t
- CPI_0 is the CPI at time 0
- μ is the drift rate of the CPI
- σ is the volatility of the CPI
- W_t is a Wiener process

This formula is used to model the CPI under geometric Brownian motion.  
Geometric Brownian motion is a stochastic process that is often used to model the prices of financial assets and other economic variables.  

The Monte Carlo method works by generating a large number of random samples from the probability distribution of the Wiener process.  
These random samples are then used to generate a large number of simulated paths for the future CPI.  
The simulated paths are then used to estimate the probability of various future events, such as the probability that the inflation rate in a given year will be below a certain target.  

In this case, the price is the CPI. The Monte Carlo method is being used to generate a large number of simulated paths for the future CPI. The simulated paths are then used to estimate the probability of various future events, such as the probability that the inflation rate in a given year will be below a certain target.

In [ ]:
n = 10
t = 10
number_simulations = 10**6

- **n** is assigned the value 10, representing the number of time steps. This is the number of discrete points in time that will be used to model the future path of the CPI.
- **t** is assigned the value 10, representing the total time period. This is the length of time over which the future path of the CPI will be modeled.
- **number_simulations** is assigned the value 10**6, representing the number of Monte Carlo simulations to be performed. This is the number of times that the future path of the CPI will be simulated.

In [ ]:
mu = inf['CPI'].mean()
sigma = inf['CPI'].std()

- **mu** is assigned the mean of the CPI column in the inf DataFrame. This is the historical mean of the percentage change in the CPI.
- **sigma** is assigned the standard deviation of the CPI column in the inf DataFrame. This is the historical standard deviation of the percentage change in the CPI.

In [ ]:
cpi_index = cpi_80.index.tolist()
v0 = cpi_80.CPI[cpi_index[-1]]

- **cpi_index** is assigned a list of the indices in the cpi_80 DataFrame.
- **v0** is assigned the value of the CPI column in the cpi_80 DataFrame at the last index in cpi_index. This is the current value of the CPI.

In [ ]:
dt = t/n
dw_ant = np.random.normal(scale=np.sqrt(dt), size=(int(number_simulations/2), n))
dw = np.concatenate((dw_ant, -dw_ant), axis=0)
dw = np.random.normal(scale=np.sqrt(dt), size=(number_simulations, n))

- **dt** is assigned the value of **t** divided by **n**, representing the time step size.
- **dw_ant** is assigned a NumPy array of normally distributed random numbers with a scale of **np.sqrt(dt)** and a size of **(int(number_simulations/2), n)**. These are the Wiener increments for the first half of the Monte Carlo simulations.
- **dw** is assigned the concatenation of **dw_ant** and its negative, along the axis 0. This ensures that the Wiener increments are antithetic, meaning that the sum of the Wiener increments over any time interval is zero.  
- **dw** is reassigned a NumPy array of normally distributed random numbers with a scale of **np.sqrt(dt)** and a size of **(number_simulations, n)**. These are the Wiener increments for all of the Monte Carlo simulations.

In [ ]:
w = np.cumsum(dw, axis=1)

- **w** is assigned the cumulative sum of **dw** along the axis 1. This is the Wiener process, which is a continuous-time stochastic process that is used to model Brownian motion.

In [ ]:
time_step = np.linspace(dt, t, n)
time_steps = np.broadcast_to(time_step, (number_simulations, n))

- **time_step** is assigned a NumPy array of linearly spaced numbers between **dt** and **t**, with **n** elements. This is the vector of time steps.
- **time_steps** is assigned the result of broadcasting time_step to a shape of **(number_simulations, n)**. This ensures that each Monte Carlo simulation has its own vector of time steps.

In [ ]:
vt = v0 * np.exp((mu - 0.5 * sigma ** 2) * time_steps + sigma * w)

- **vt** is assigned the value of v0 multiplied by the exponential of **(mu - 0.5 * sigma ** 2) * time_steps + sigma * w**. This is the solution to the stochastic differential equation for the geometric Brownian motion model.

In [ ]:
vt = np.insert(vt, 0, v0, axis=1)

- **vt** is reassigned the result of inserting v0 into the first column of vt. This ensures that the first value in each simulated path is the current value of the CPI.

In [ ]:
cpi_index = cpi_80.index.tolist()
index = range(cpi_index[-1], cpi_index[-1] + n + 1)

- **index** is assigned a range of integers from the last index in cpi_index to the last index in cpi_index plus n plus 1. This is the index for the simulated paths.

In [ ]:
paths = pd.DataFrame(np.transpose(vt), index=index)
mean = paths.mean(axis=1)

- **paths** is assigned a Pandas DataFrame with the transposed values of vt as the data and index as the index. This is the DataFrame of simulated paths for the CPI.
- **mean** is assigned the mean of paths along the axis 1. This is the mean simulated path for the CPI.

In [ ]:
paths.head(11)

The following plot shows the range of possible future paths for the CPI.  
- The 80th percentile represents the value above which 80% of the simulated paths fall.
- The 50th percentile represents the median value of the simulated paths.
- The 20th percentile represents the value below which 20% of the simulated paths fall.

The red line in the plot shows the mean of the simulated paths. The mean is the average value of all of the simulated paths.  
The green and blue lines in the plot show the 20th and 80th percentiles of the simulated paths.  

The area between the green and blue lines represents the range of values within which 60% of the simulated paths fall.  

This plot can be used to visualize the range of possible future paths for the CPI.  
The plot can also be used to estimate the probability of various future events.  
For example, the probability that the CPI will be above a certain value in a given year can be estimated by looking at the percentage of simulated paths that are above that value.  

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
paths.quantile(0.8, axis=1).plot(figsize=(20, 8))
paths.mean(axis=1).plot(color='red')
paths.quantile(0.2, axis=1).plot(color='green')

In [ ]:
predicted_inf = paths.pct_change().dropna()
predicted_inf.head(11)

- **predicted_inf** is assigned the percentage change of the paths DataFrame, with any missing values dropped. This is the DataFrame of simulated paths for the inflation rate.

In [ ]:
inf_2025 = predicted_inf.loc[2025, :].values
print("Estimated probability that the inflation rate in 2025 will be less than 2% in percetage: ", np.count_nonzero(inf_2025 < 0.02) / 1e6 * 100)

**inf_2025** is assigned the values of the predicted_inf DataFrame at the index 2025. This is the vector of simulated inflation rates for the year 2025.  
The number of simulated inflation rates for the year 2025 that are less than 2% is counted.  
The count is divided by the total number of simulated inflation rates and multiplied by 100 to express the result as a percentage.  
This is the estimated probability that the inflation rate in 2025 will be less than 2%.  